In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scikit-learn nos ofrece una variedad ampliada de modelos Naive Bayes, para este problema usamos MultinomialNB que es pensado para este tipo de problemas
from sklearn.naive_bayes import MultinomialNB   

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

# TP3: Detector de SPAM

Uno de los problemas más comunes en la clasificación es la detección de correos electrónicos SPAM. Uno de los primeros modelos utilizados para abordar este problema fue el clasificador de Bayes ingenuo. La detección de SPAM es un problema persistente en el mundo digital, ya que los spammers continúan adaptando sus estrategias para eludir los filtros de correo no deseado. Además del clasificador de Bayes ingenuo, se han desarrollado y utilizado una variedad de técnicas más avanzadas en la detección de SPAM, que incluyen algoritmos de aprendizaje automático, redes neuronales y métodos basados en reglas.

En este trabajo práctico, utilizaremos un conjunto de datos que consta de 4601 observaciones de correos electrónicos, de los cuales 2788 son correos legítimos y 1813 son correos SPAM. Dado que el contenido de los correos electrónicos es un tipo de dato no estructurado, es necesario procesarlo de alguna manera. Para este conjunto de datos, ya se ha aplicado un procesamiento típico en el Procesamiento del Lenguaje Natural (NLP), que consiste en contar la frecuencia de palabras observadas en los correos.

El procesamiento de lenguaje natural (NLP) desempeña un papel fundamental en la detección de SPAM, ya que permite analizar el contenido de los correos electrónicos y extraer características relevantes para la clasificación. Además de contar la frecuencia de palabras, se pueden utilizar técnicas más sofisticadas, como la extracción de características semánticas y el análisis de sentimientos, para mejorar la precisión de los modelos de detección de SPAM.

En este proceso, se cuenta la cantidad de ocurrencias de cada palabra en los diferentes correos.

![spam counter](./spam.png)

Con el fin de preservar la privacidad de los mensajes, la frecuencia de palabras se encuentra normalizada. El conjunto de datos está compuesto por 54 columnas de atributos que se denominan:

- `word_freq_XXXX`: Donde `XXXX` es la palabra o símbolo. Los valores son enteros que van de 0 a 20k.

Además, hay una columna adicional llamada `spam`, que es 1 si el correo es SPAM o 0 si no lo es.

Los clasificadores de Bayes ingenuos fueron los primeros filtros utilizados por las aplicaciones de correo electrónico, basados en este principio de palabras. La idea es que, partiendo de un dato a priori sobre la probabilidad de que un correo sea SPAM o no, ciertas palabras nos indicarán que la probabilidad a posteriori, dadas esas palabras, es más probable que el correo sea SPAM o no.

In [2]:
dataset = pd.read_csv("dataset/spambase.csv", header=None) # cargando los datos desde un CSV
dataset.head(10)
column_names = [
    "word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", "word_freq_our",
    "word_freq_over", "word_freq_remove", "word_freq_internet", "word_freq_order", "word_freq_mail",
    "word_freq_receive", "word_freq_will", "word_freq_people", "word_freq_report", "word_freq_addresses",
    "word_freq_free", "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit",
    "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", "word_freq_hp",
    "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs",
    "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85",
    "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct",
    "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project", "word_freq_re",
    "word_freq_edu", "word_freq_table", "word_freq_conference",  # Estas son las primeras 48 (0-47)
    "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_#",  # (48-53)
    "capital_run_length_average", "capital_run_length_longest", "capital_run_length_total", # (54-56)
    "spam" # (57)
]

dataset.columns = column_names

print(dataset.head())

   word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0            0.00               0.64           0.64           0.0   
1            0.21               0.28           0.50           0.0   
2            0.06               0.00           0.71           0.0   
3            0.00               0.00           0.00           0.0   
4            0.00               0.00           0.00           0.0   

   word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0           0.32            0.00              0.00                0.00   
1           0.14            0.28              0.21                0.07   
2           1.23            0.19              0.19                0.12   
3           0.63            0.00              0.31                0.63   
4           0.63            0.00              0.31                0.63   

   word_freq_order  word_freq_mail  ...  char_freq_;  char_freq_(  \
0             0.00            0.00  ...         0.00        0.000   
1 

In [3]:
print(dataset.columns)



Index(['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
       'word_freq_our', 'word_freq_over', 'word_freq_remove',
       'word_freq_internet', 'word_freq_order', 'word_freq_mail',
       'word_freq_receive', 'word_freq_will', 'word_freq_people',
       'word_freq_report', 'word_freq_addresses', 'word_freq_free',
       'word_freq_business', 'word_freq_email', 'word_freq_you',
       'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000',
       'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george',
       'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
       'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
       'word_freq_technology', 'word_freq_1999', 'word_freq_parts',
       'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting',
       'word_freq_original', 'word_freq_project', 'word_freq_re',
       'word_freq_edu', 'word_freq_table', 'word_freq_conference',


Para obtener las palábras más usadas podemos hacer un `groupby`:

In [4]:
column_sum = dataset.groupby(by="spam", as_index=False).sum()
print(column_sum)

   spam  word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0     0          204.86             681.57         559.22          2.47   
1     1          276.19             298.51         732.08        298.55   

   word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0         504.74          124.19             26.16              107.10   
1         931.80          317.05            499.31              377.36   

   word_freq_order  ...  word_freq_conference  char_freq_;  char_freq_(  \
0           106.08  ...                142.82      140.183      442.116   
1           308.32  ...                  3.81       37.299      197.563   

   char_freq_[  char_freq_!  char_freq_$  char_freq_#  \
0       63.242      306.634       32.476       60.536   
1       14.864      931.361      316.329      143.004   

   capital_run_length_average  capital_run_length_longest  \
0                    6627.915                       50782   
1                   17258.246  

Y despues se pueden combinar las columnas en usando [pd.melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html)

In [5]:
# Obtenemos los atributos y target
X = (dataset.drop(columns="spam") * 100).astype(int)
print (X)
#X = dataset2.drop(columns="spam")
y = dataset["spam"]
print (y)

      word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0                  0                 64             64             0   
1                 21                 28             50             0   
2                  6                  0             71             0   
3                  0                  0              0             0   
4                  0                  0              0             0   
...              ...                ...            ...           ...   
4596              31                  0             62             0   
4597               0                  0              0             0   
4598              30                  0             30             0   
4599              96                  0              0             0   
4600               0                  0             65             0   

      word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0                32               0                 0     

Se separa el dataset en entrenamiento y evaluación

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3)
print("X_train:")
print(X_train)
print("y_train:")
print(y_train)

print("X_test:")
print(X_test)
print("y_test:")
print(y_test)

X_train:
      word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
3636               0                  0              0             0   
4146               0                  0              0             0   
1439               0                  0              0             0   
516               26                 72             85             0   
4431               0                  0             40             0   
...              ...                ...            ...           ...   
952                8                  8             76             0   
888               34                  0            170             0   
4012               0                  0             13             0   
3754              33                  0              0             0   
3787               0                  0              0             0   

      word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
3636              0               0              

Escalamos para aplicar en regresión logística

In [8]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

### las top 10 palabras en spam:

In [11]:
# Filtramos la fila de spam=1 y quitamos la columna 'spam'
spam_sum = column_sum[column_sum["spam"] == 1].drop(columns="spam")
#print(spam_sum)

# Trasponemos para tener las columnas como filas y poder ordenar
spam_sum_t = spam_sum.T
#Sprint(spam_sum_t.columns)
top10_spam_words = spam_sum_t.iloc[:, 0].sort_values(ascending=False).head(10)
print("Top 10 palabras en SPAM:")
print(top10_spam_words)
"""
# Ordenamos por el valor de la frecuencia total
top10_spam_words = spam_sum_t[0].sort_values(ascending=False).head(10)

"""

Top 10 palabras en SPAM:
capital_run_length_total      853233.000
capital_run_length_longest    189265.000
capital_run_length_average     17258.246
word_freq_you                   4105.610
word_freq_your                  2502.610
word_freq_will                   997.100
word_freq_free                   939.790
word_freq_our                    931.800
char_freq_!                      931.361
word_freq_all                    732.080
Name: 1, dtype: float64


'\n# Ordenamos por el valor de la frecuencia total\ntop10_spam_words = spam_sum_t[0].sort_values(ascending=False).head(10)\n\n'

In [12]:
### las top 10 palabras en spam

In [13]:
nonspam_sum = column_sum[column_sum["spam"] == 0].drop(columns="spam")
nonspam_sum_t = nonspam_sum.T
top10_nonspam_words = nonspam_sum_t[0].sort_values(ascending=False).head(10)
print("Top 10 palabras en NO SPAM:")
print(top10_nonspam_words)


Top 10 palabras en NO SPAM:
capital_run_length_total      450181.000
capital_run_length_longest     50782.000
capital_run_length_average      6627.915
word_freq_you                   3541.710
word_freq_george                3527.560
word_freq_hp                    2496.580
word_freq_will                  1495.270
word_freq_your                  1223.100
word_freq_hpl                   1204.400
word_freq_re                    1159.140
Name: 0, dtype: float64
